# Generate a reference document matching LSOAs to Police Force codes

Some data is retrieved by link shown in the code.

Other data was downloaded from data.police for all forces for 2019-06 and uploaded into the colabs environment.

In [0]:
import pandas as pd
import glob

In [3]:
# get sheet with LSOA/MSOA/LA breakdown
! wget https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales/rft-table-php01-2011-msoas-and-lsoas.zip

--2019-08-12 11:29:51--  https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales/rft-table-php01-2011-msoas-and-lsoas.zip
Resolving www.ons.gov.uk (www.ons.gov.uk)... 104.20.61.76, 104.20.60.76, 2606:4700:10::6814:3c4c, ...
Connecting to www.ons.gov.uk (www.ons.gov.uk)|104.20.61.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘file?uri=%2Fpeoplepopulationandcommunity%2Fpopulationandmigration%2Fpopulationestimates%2Fdatasets%2F2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales%2Frft-table-php01-2011-msoas-and-lsoas.zip’

file?uri=%2Fpeoplep     [     <=>            ]   2.87M  2.59MB/s    in 1.1s    

2019-08-12 11:29:53 (2.59 MB/s) - ‘file?uri=%2Fpeoplepopulationandcommunity%2Fpopulationandmigration%2Fpopulationestimates%2Fdatasets%2F2011censuspopulati

In [4]:
# unzip file
! unzip file?uri\=%2Fpeoplepopulationandcommunity%2Fpopulationandmigration%2Fpopulationestimates%2Fdatasets%2F2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales%2Frft-table-php01-2011-msoas-and-lsoas.zip

Archive:  file?uri=%2Fpeoplepopulationandcommunity%2Fpopulationandmigration%2Fpopulationestimates%2Fdatasets%2F2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales%2Frft-table-php01-2011-msoas-and-lsoas.zip
  inflating: R1_4_EW__RT__Table_PHP01___LSOA_MSOA_v4.xls  


In [0]:
# open file on sheet five skipping useless rows
ONS_geo_ref = pd.read_excel('R1_4_EW__RT__Table_PHP01___LSOA_MSOA_v4.xls', sheet_name=5, skiprows=11)

In [4]:
ONS_geo_ref.columns

Index(['Region code', 'Region name', 'Local authority code',
       'Local authority name', 'MSOA Code', 'MSOA Name', 'LSOA Code',
       'LSOA Name', 'Unnamed: 8', 'Persons', 'Persons.1', 'Persons.2',
       'Unnamed: 12', 'Hectares', 'Persons per hectare', 'Unnamed: 15',
       'Households', 'Persons per household'],
      dtype='object')

In [0]:
# select out desired columns
ONS_geo_ref1 = ONS_geo_ref[['Region code', 'Region name', 'Local authority code',
       'Local authority name', 'MSOA Code', 'MSOA Name', 'LSOA Code',
       'LSOA Name']]

In [6]:
# drop empty rows
ONS_geo_ref1.dropna(how='all', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
# using data.police data concatenate all files into one master file
file_to_concat = []

for file in glob.glob('/content/*.csv'):
  
  frame = pd.read_csv(file)
  
  file_to_concat.append(frame)
  
complete_frame = pd.concat(file_to_concat)

In [63]:
# view shape
complete_frame.shape

(566611, 12)

In [0]:
# create a working frame from the master frame

# group data by force showing counts of LSOA codes (we just want force:LSOA codes)
group_frame = complete_frame.groupby('Falls within')['LSOA code'].value_counts().reset_index(['Falls within'])

# rename columns after index reset
group_frame.columns = ['Police_force','Counts']

# reset index to get LSOA code col back
group_frame.reset_index('LSOA code', inplace=True)

# select out only LSOA codes and police force
group_frame = group_frame[['LSOA code','Police_force']]

In [0]:
# exclude all incidents allocated to BTP 
# we are not interested in this we want general geographical link between force and LSOA
group_frame = group_frame[group_frame.Police_force != 'British Transport Police']

In [33]:
group_frame.head()

,LSOA code,Police_force
0,E01014540,Avon and Somerset Constabulary
1,E01033342,Avon and Somerset Constabulary
2,E01014801,Avon and Somerset Constabulary
3,E01029284,Avon and Somerset Constabulary
4,E01029112,Avon and Somerset Constabulary


In [34]:
# check we can match LSOA code 
ONS_geo_ref[ONS_geo_ref['LSOA Code'].isin(['E01029112'])]

,Region code,Region name,Local authority code,Local authority name,MSOA Code,MSOA Name,LSOA Code,LSOA Name,Unnamed: 8,Persons,Persons.1,Persons.2,Unnamed: 12,Hectares,Persons per hectare,Unnamed: 15,Households,Persons per household
32628,E12000009,SOUTH WEST,E07000188,Sedgemoor,E02006069,Sedgemoor 009,E01029112,Sedgemoor 009E,NaN,1876,1828,48,NaN,47,40.2,NaN,1007,1.8


In [0]:
# build a list of local authorities based on the LSOA code from groupby frame created above based on the ONS reference sheet

LA_name_lst = []

for cd in group_frame['LSOA code']:
  
  if len(ONS_geo_ref[ONS_geo_ref['LSOA Code'].isin([cd])]) > 0:
    
    LA_name_lst.append(ONS_geo_ref[ONS_geo_ref['LSOA Code'].isin([cd])]['Local authority name'].tolist()[0])

In [0]:
# add this column so we now have LSOA code, local authority name and police force 
group_frame['LA_name'] = LA_name_lst

In [37]:
group_frame.head()

,LSOA code,Police_force,LA_name
0,E01014540,Avon and Somerset Constabulary,"Bristol, City of UA"
1,E01033342,Avon and Somerset Constabulary,"Bristol, City of UA"
2,E01014801,Avon and Somerset Constabulary,North Somerset UA
3,E01029284,Avon and Somerset Constabulary,Taunton Deane
4,E01029112,Avon and Somerset Constabulary,Sedgemoor


In [0]:
# identify missing LSOAs from groupby frame using the reference frame with all LSOA names 

missing_LSOA = ONS_geo_ref[~ONS_geo_ref['LSOA Code'].isin(group_frame['LSOA code'])][['LSOA Code','Local authority name']]

# some NAs still in reference frame so dropped
missing_LSOA.dropna(how='any', inplace=True)

In [0]:
# using the local authority names from missing LSOA data captured above match local authority names to LA names in groupby frame
# if match extract police_force and add it to list

missing_constab = []

for LA in missing_LSOA['Local authority name']:
  
  missing_constab.append(group_frame[group_frame['LA_name'].isin([LA])]['Police_force'].tolist()[0])

In [0]:
# add these constabulary names to the missing lsoa data
missing_LSOA['Police_force'] = missing_constab

In [0]:
# rename columns so they match for concatenation

missing_LSOA.columns = ['LSOA_code', 'LA_name', 'Police_force']

group_frame.columns = ['LSOA_code', 'Police_force', 'LA_name']

In [53]:
# concatenate date into combined frame and reset index

combined_frame = pd.concat([group_frame, missing_LSOA])

combined_frame.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [55]:
# check the shape expecting 1388
combined_frame[combined_frame.LA_name.isin(['Leeds','Kirklees','Bradford','Wakefield','Calderdale'])].shape

(1413, 3)

In [62]:
# some crossovers discovered
combined_frame[combined_frame.LSOA_code == 'E01029131']

,LA_name,LSOA_code,Police_force
67,Sedgemoor,E01029131,Avon and Somerset Constabulary
24890,Sedgemoor,E01029131,Nottinghamshire Police


In [0]:
# will arbitrarily drop duplicates except the first instance 
combined_frame_ = combined_frame.drop_duplicates(subset='LSOA_code', keep='first')

In [66]:
# check the shape expecting 1388
combined_frame_[combined_frame_.LA_name.isin(['Leeds','Kirklees','Bradford','Wakefield','Calderdale'])].shape

(1388, 3)

In [0]:
# now happy to proceed with saving this list to use as reference list between LSOA and police force
combined_frame_.to_csv('PoliceforceLSOA.csv')

In [0]:
# prompt download from colab
from google.colab import files

files.download('PoliceforceLSOA.csv')